In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.42 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt
Child returned 0


rm -rf /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD635
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


Child returned 1
0.36 seconds.
File does not exist: /shared/CSHL_data_processed/MD635/MD635_alignedTo_MD635-F63-2016.05.19-08.39.03_MD635_2_0188_cropbox.txt


In [3]:
dataset_settings

,network_model,stain,margins,num_sample_per_class,stacks,cell_features_used
dataset_id,,,,,,
20,Inception-BN,nissl,200/500,1000,MD585,NaN
21,Inception-BN,nissl,200/500,1000,MD589,NaN
22,Inception-BN,nissl,200/500,1000,MD594,NaN
90,NaN,nissl,500,1000,MD589,allSizeHist/huMomentsHist
92,NaN,nissl,500,1000,MD589,largeLargeLinkLenHist
93,NaN,nissl,500,1000,MD594,largeLargeLinkLenHist
94,NaN,nissl,500,1000,MD589,largeSizeHist
95,NaN,nissl,500,1000,MD594,largeSizeHist
96,NaN,nissl,500,1000,MD589,largeSizeHist/largeLargeLinkLenHist


In [4]:
dataset_id = 20
dataset_properties = dataset_settings.loc[dataset_id]

num_samples_per_label = dataset_properties['num_sample_per_class']
stacks = dataset_properties['stacks'].split('/')

In [5]:
structures_to_sample = all_known_structures

# negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

# margins_to_sample = map(int, str(dataset_properties['margins']).split('/'))
margins_to_sample = [500]

surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                             for m in margins_to_sample
                             for s in structures_to_sample 
                             for surr_l in structures_to_sample
                             if surr_l != s]

surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                             for m in margins_to_sample
                             for s in structures_to_sample]

labels_to_sample = structures_to_sample + \
surround_positive_labels_to_sample + surround_noclass_labels_to_sample + \
['noclass']

In [11]:
addresses = generate_patch_dataset(num_samples_per_label=num_samples_per_label, 
                                                     stacks=stacks,
                                                     labels_to_sample=labels_to_sample)

Read: 2.65 seconds
Cell is empty.
Cell is empty.
Sample addresses (stack MD585): 4. seconds.
Sample addresses: 6.96 seconds


# Load Features

In [ ]:
features, addresses, labels_found = generate_dataset(num_samples_per_label=num_samples_per_label, 
                                                     stacks=stacks,
                                                     labels_to_sample=labels_to_sample)

In [ ]:
# for label in sorted(features.iterkeys()):
#     print label, len(addresses[label])

In [ ]:
# Save training features
features_fp = os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_features.hdf')
create_parent_dir_if_not_exists(features_fp)
save_hdf_v2(features, features_fp)
upload_from_ec2_to_s3(features_fp)
# train_feat_dir = create_if_not_exists(os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset, 'patch_features'))
# for label, feats in training_features.iteritems():
#     bp.pack_ndarray_file(feats, os.path.join(train_feat_dir, label + '.bp'))

# Save training addresses
addresses_fp = os.path.join(CLF_ROOTDIR, 'datasets', 'dataset_%d' % dataset_id, 'patch_addresses.pkl')
save_pickle(addresses, addresses_fp)
upload_from_ec2_to_s3(addresses_fp)

# Gather patches

In [7]:
! mkdir /tmp/patches_dataset_20

mkdir: cannot create directory ‘/tmp/patches_dataset_20’: File exists


In [20]:
patches = apply_function_to_dict(lambda addrs: extract_patches_given_locations_multiple_sections(addrs, 'grid_index', version='cropped_gray'), 
                   addresses)

In [23]:
for label, ps in patches.iteritems():
    print label
    bp.pack_ndarray_file(np.array(ps), '/tmp/patches_dataset_20/%s.bp' % label)

Amb_surround_500_7N
12N_surround_500_AP
SNC_surround_500_SNR
SNR_surround_500_SNC
SNR_surround_500_PBG
5N
Sp5I_surround_500_Sp5C
VCP_surround_500_noclass
Sp5I_surround_500_Sp5O
6N_surround_500_7n
12N
3N_surround_500_noclass
LRt_surround_500_Amb
Sp5O
Pn_surround_500_noclass
Sp5I
VCA_surround_500_noclass
Sp5C
3N_surround_500_4N
SNC
SC_surround_500_IC
LC_surround_500_noclass
4N_surround_500_3N
DC_surround_500_VCA
SNR
DC_surround_500_VCP
noclass
SNC_surround_500_noclass
RtTg
Pn
10N
VLL
RtTg_surround_500_Tz
IC_surround_500_noclass
PBG_surround_500_noclass
12N_surround_500_noclass
Pn_surround_500_Tz
PBG_surround_500_SNR
Sp5O_surround_500_Sp5I
PBG
LRt
RMC_surround_500_SNR
SNR_surround_500_RMC
Sp5C_surround_500_noclass
AP_surround_500_10N
7N_surround_500_Amb
5N_surround_500_noclass
RtTg_surround_500_noclass
Pn_surround_500_RtTg
VCA_surround_500_DC
7n_surround_500_5N
Sp5I_surround_500_noclass
AP_surround_500_12N
Amb
RMC
7n_surround_500_6N
10N_surround_500_12N
SNR_surround_500_VLL
RMC_surround_5

In [24]:
! aws s3 cp --recursive /tmp/patches_dataset_20/ s3://mousebrainatlas-data/patches_dataset_20

upload: ../../../tmp/patches_dataset_20/10N_surround_500_AP.bp to s3://mousebrainatlas-data/patches_dataset_20/10N_surround_500_AP.bp
upload: ../../../tmp/patches_dataset_20/10N.bp to s3://mousebrainatlas-data/patches_dataset_20/10N.bp
upload: ../../../tmp/patches_dataset_20/10N_surround_500_12N.bp to s3://mousebrainatlas-data/patches_dataset_20/10N_surround_500_12N.bp
upload: ../../../tmp/patches_dataset_20/10N_surround_500_noclass.bp to s3://mousebrainatlas-data/patches_dataset_20/10N_surround_500_noclass.bp
upload: ../../../tmp/patches_dataset_20/12N.bp to s3://mousebrainatlas-data/patches_dataset_20/12N.bp
upload: ../../../tmp/patches_dataset_20/12N_surround_500_AP.bp to s3://mousebrainatlas-data/patches_dataset_20/12N_surround_500_AP.bp
upload: ../../../tmp/patches_dataset_20/12N_surround_500_10N.bp to s3://mousebrainatlas-data/patches_dataset_20/12N_surround_500_10N.bp
upload: ../../../tmp/patches_dataset_20/12N_surround_500_noclass.bp to s3://mousebrainatlas-data/patches_dataset

upload: ../../../tmp/patches_dataset_20/SNR_surround_500_RMC.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR_surround_500_RMC.bp
upload: ../../../tmp/patches_dataset_20/SNR_surround_500_PBG.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR_surround_500_PBG.bp
upload: ../../../tmp/patches_dataset_20/SNR_surround_500_VLL.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR_surround_500_VLL.bp
upload: ../../../tmp/patches_dataset_20/SNC_surround_500_noclass.bp to s3://mousebrainatlas-data/patches_dataset_20/SNC_surround_500_noclass.bp
upload: ../../../tmp/patches_dataset_20/SNR.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR.bp
upload: ../../../tmp/patches_dataset_20/SNR_surround_500_SNC.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR_surround_500_SNC.bp
upload: ../../../tmp/patches_dataset_20/SNR_surround_500_noclass.bp to s3://mousebrainatlas-data/patches_dataset_20/SNR_surround_500_noclass.bp
upload: ../../../tmp/patches_dataset_20/Sp5C.bp to s3://mousebrain